In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import ipywidgets as widgets
from IPython.display import clear_output
import os

#Seccion de API google Sheet
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from googleapiclient.discovery import build
#Subida de archivo
from openpyxl import Workbook
from openpyxl.drawing.image import Image

## **Seccion 1:** Conexion con la API google Sheet
****

In [ ]:
#informacion basica
Scopes = ['https://www.googleapis.com/auth/spreadsheets']
key = 'key.json'
id_sheet = '1v4w_c-0D8moJFnzxFxrCGN0d7Hyqni8HkgCCTD5e5F4'

#conectar
creds = None
creds = service_account.Credentials.from_service_account_file(key,scopes=Scopes)

#Generar API
service = build('sheets','v4',credentials=creds)
EXCEl = service.spreadsheets()

In [ ]:
res = EXCEl.values().get(spreadsheetId=id_sheet,range='main_tables!A1:l370000').execute()

res.get('values',[])[0]

## **Seccion 2:** Cargue de los datos presentes en google sheet
****

In [ ]:
datos =pd.DataFrame(res.get('values',[])[1:],columns=res.get('values',[])[0])
datos.head()

## **Seccion 3**: Analisis de los datos
***
En esta seccion analizaremos los datos presentes en el google sheet e intentaremos responder algunas cuestiones como:
 

### **Seccion 3.0**:filtros
****

In [ ]:
#filtro de municipio de origen
Municipios_origen=widgets.TagsInput(value=['CALI'],allowed_tags=list(datos['MUNICIPIO_ORIGEN_RUTA'].unique()),allow_duplicates=False)

mes_despacho = widgets.TagsInput(
    value=[str(x) for x in range(1, 13)],
    allowed_tags=[str(x) for x in range(1, 13)],
    allow_duplicates=False,
    description='Mes Despacho:',
)
year = widgets.IntRangeSlider(value=[datos['ano_despacho'].unique().min(),datos['ano_despacho'].unique().max()],min=datos['ano_despacho'].unique().min(),max=datos['ano_despacho'].unique().max())

clase_vehiculo=widgets.TagsInput(value=list(datos['CLASE_VEHICULO'].unique()),allowed_tags=list(datos['CLASE_VEHICULO'].unique()),allow_duplicates=False)



### **Seccion 3.1**: Tendencia de los viajes en carretera
***

In [ ]:
 def Tendencia_viajes(municipios,mes,year,clase_vehi):
    #creacion del filtro
    datos_filtrados=datos.loc[(datos['MUNICIPIO_ORIGEN_RUTA'].isin(municipios)) & (datos['CLASE_VEHICULO'].isin(clase_vehi)) &(datos['mes_despacho'].isin(mes))  &(datos['ano_despacho'].between(str(year[0]),str(year[-1])))].groupby('MUNICIPIO_DESTINO_RUTA')['MUNICIPIO_DESTINO_RUTA'].count()
    #extraccion de porcentajes
    datos_filtrados = datos_filtrados/100
    # las 10 mejores rutas
    top_10 = datos_filtrados.nlargest(10)

    #limieza de pantalla
    clear_output(wait=True)


    print(f"Desviacion estandar: {top_10.std():.2f}")
    plt.figure(figsize=(20, 5))
    sns.lineplot(top_10,x='MUNICIPIO_DESTINO_RUTA',y=top_10,marker='o', markersize=8)
    
    for y,x in zip(top_10,top_10.keys()):
      plt.text(x,y,f'{y-top_10.std():.2f}%',ha='center',va='bottom',fontsize=10)

    plt.xlabel('Municipio Destino')
    plt.ylabel('Porcentaje')
    plt.title(f'Histograma para Municipio de origen: {", ".join(municipios)}')
    plt.grid(True)

    
    #Guardar grafico
    ruta = f'Img_analisis/Tendencia_viaje.png'
    plt.savefig(ruta)

    plt.show()


#preparar conexion
widgets.interactive(Tendencia_viajes,municipios=Municipios_origen,mes=mes_despacho,year=year,clase_vehi=clase_vehiculo)

### **Seccion 3.2**: Porcentaje de vehiculos presentes en el municipio de destino
****

In [ ]:
def Vehiculos_municipio(municipios,mes,year,clase_vehi):
    # Sacar filtro
    datos_filtrados_vehi = datos.loc[(datos['MUNICIPIO_ORIGEN_RUTA'].isin(municipios)) & (datos['CLASE_VEHICULO'].isin(clase_vehi)) &(datos['mes_despacho'].isin(mes))  &(datos['ano_despacho'].between(str(year[0]),str(year[-1])))]
    datos_filtrados = datos.loc[(datos['MUNICIPIO_ORIGEN_RUTA'].isin(municipios)) & (datos['CLASE_VEHICULO'].isin(clase_vehi)) &(datos['mes_despacho'].isin(mes))  &(datos['ano_despacho'].between(str(year[0]),str(year[-1])))].groupby('MUNICIPIO_DESTINO_RUTA')['MUNICIPIO_DESTINO_RUTA'].count()

    # las 10 mejores rutas
    datos_filtrados_vehi = datos_filtrados_vehi.pivot_table(index='MUNICIPIO_DESTINO_RUTA', columns='CLASE_VEHICULO', aggfunc='size', fill_value=0)
    top_10 = datos_filtrados.nlargest(10)
    top_10_vehi = datos_filtrados_vehi.sort_values(by=clase_vehi[0], ascending=False)[:10]
    

    print("Valores de la desviacion estandar")
    print(pd.DataFrame(top_10_vehi).std() / len(top_10_vehi))
    clear_output(wait=True)

    # Creacion de grafica
    plt.figure(figsize=(20, 5))
    for column in clase_vehi:
        if column in top_10_vehi:
            plt.bar(top_10.keys(), top_10_vehi[column],label=column)

    plt.legend()
    plt.xlabel('Municipio Destino')
    plt.ylabel('Porcentaje')
    plt.title(f'Numero de vehiculos que salieron de - {", ".join(municipios)}-')
    plt.grid(True)
    
    #Guardar grafico
    ruta = f'Img_analisis/Vehiculos_municipio_des.png'
    plt.savefig(ruta)

    plt.show()



#mostrar datos
widgets.interactive(Vehiculos_municipio,municipios=Municipios_origen,mes=mes_despacho,year=year,clase_vehi=clase_vehiculo)

### **Seccio 3.3:** Estimacion hora-destino
****

In [ ]:
def Hora_destino(municipios,mes,year,clase_vehi):
    #Crear filtro
    datos_filtrados=datos.loc[(datos['MUNICIPIO_ORIGEN_RUTA'].isin(municipios)) & (datos['CLASE_VEHICULO'].isin(clase_vehi)) &(datos['mes_despacho'].isin(mes))  &(datos['ano_despacho'].between(str(year[0]),str(year[-1])))].groupby('MUNICIPIO_DESTINO_RUTA')['HORA_DESPACHO'].std()
    
    top_10 = datos_filtrados.nlargest(10)


    print(f"Desviacion estandar: {top_10.std():.2f}")
    plt.figure(figsize=(20, 5))
    sns.lineplot(top_10,x='MUNICIPIO_DESTINO_RUTA',y=top_10,marker='o', markersize=8)
    
    for y,x in zip(top_10,top_10.keys()):
      plt.text(x,y,f'{y-top_10.std():.1f} horas',ha='center',va='bottom',fontsize=10)

    plt.xlabel('Municipio Destino')
    plt.ylabel("Estimacion de tiempo de llegada")
    plt.title(f' Promedio de duracion del viaje dado que saliste de  -{", ".join(municipios)}-')
    plt.grid(True)
    
    #Guardar grafico
    ruta = f'Img_analisis/Hora_des.png'
    plt.savefig(ruta)
    plt.show()

widgets.interactive(Hora_destino,municipios=Municipios_origen,mes=mes_despacho,year=year,clase_vehi=clase_vehiculo)
    

### **Seccion 3.4:** Distribucion de personas por terminal
****

In [ ]:
def Dist_Terminal(municipios,mes,year,clase_vehi):
    #Crear filtro
    datos_filtrados=datos.loc[(datos['TIPO_DESPACHO'].isin(['ORIGEN']))&(datos['MUNICIPIO_ORIGEN_RUTA'].isin(municipios)) & (datos['CLASE_VEHICULO'].isin(clase_vehi)) &(datos['mes_despacho'].isin(mes))  &(datos['ano_despacho'].between(str(year[0]),str(year[-1])))].groupby('TERMINAL')['PASAJEROS'].count()

    top_10 = datos_filtrados.nlargest(10)
    clear_output(wait=True)
    print(top_10)
    plt.figure(figsize=(20, 5))
    plt.bar(top_10.keys(),top_10)
    plt.title(f' Numero de personas presentes en los reminales del municipio de: -{", ".join(municipios)}-')
    plt.xlabel("Terminales")
    plt.ylabel("Cantidad de personas")

    #Guardar grafico
    ruta = f'Img_analisis/Dis_terminal.png'
    plt.savefig(ruta)
    plt.show()

    
widgets.interactive(Dist_Terminal,municipios=Municipios_origen,mes=mes_despacho,year=year,clase_vehi=clase_vehiculo)

### **Seccion 3.5:** Numero de vehiculos que salieron del terminal segun su municipio de origen
***

In [ ]:
def Vehi_ter(municipios,mes,year,clase_vehi):
        # Sacar filtro
    datos_filtrados_vehi = datos.loc[(datos['MUNICIPIO_ORIGEN_RUTA'].isin(municipios)) & (datos['CLASE_VEHICULO'].isin(clase_vehi)) &(datos['mes_despacho'].isin(mes))  &(datos['ano_despacho'].between(str(year[0]),str(year[-1])))]
    datos_filtrados = datos.loc[(datos['MUNICIPIO_ORIGEN_RUTA'].isin(municipios)) & (datos['CLASE_VEHICULO'].isin(clase_vehi)) &(datos['mes_despacho'].isin(mes))  &(datos['ano_despacho'].between(str(year[0]),str(year[-1])))].groupby('TERMINAL')['PASAJEROS'].count()

    # las 10 mejores rutas
    datos_filtrados_vehi = datos_filtrados_vehi.pivot_table(index='TERMINAL', columns='CLASE_VEHICULO', aggfunc='size', fill_value=0)
    top_10 = datos_filtrados.nlargest(10)
    top_10_vehi = datos_filtrados_vehi.sort_values(by=clase_vehi[0], ascending=False)[:10]
    

    print("Valores de la desviacion estandar")
    print(pd.DataFrame(top_10_vehi).std() / len(top_10_vehi))
    clear_output(wait=True)


    # Creacion de grafica
    plt.figure(figsize=(25, 5))
    for column in clase_vehi:
        if column in top_10_vehi:
            plt.bar(top_10.keys(), top_10_vehi[column], label=column)

    plt.legend()
    plt.xlabel('Terminal de origen')
    plt.ylabel('Cantidad')
    plt.title(f'Numero de vehiculos que salen \nde la terminal de origen en el municipio: {", ".join(municipios)}')
    plt.grid(True)

    #Guardar grafico
    ruta = f'Img_analisis/Salida_terminal.png'
    plt.savefig(ruta)
    plt.show()


widgets.interactive(Vehi_ter,municipios=Municipios_origen,mes=mes_despacho,year=year,clase_vehi=clase_vehiculo)

## **Seccion 4:** Subir los graficos a google sheet
***

In [ ]:

# Ruta local a la imagen
for path ,_,files in os.walk('Img_analisis'):
    for file in files:
        ruta_imagen = f'{path}/{file}'
        print(file[:-4])


In [ ]:
import os
from openpyxl import Workbook
from openpyxl.drawing.image import Image

# Ruta del directorio que contiene las imágenes
directorio_imagenes = 'Img_analisis'

# Crear un nuevo libro de Excel
libro_excel = Workbook()

# Recorrer todas las imágenes en el directorio
for path, _, files in os.walk(directorio_imagenes):
    for file in files:
        ruta_imagen = os.path.join(path, file)

        # Crea una nueva hoja en el libro de Excel
        nombre_nueva_hoja = file[:-4]
        hoja_nueva = libro_excel.create_sheet(title=nombre_nueva_hoja)

        # Insertar la imagen en una celda específica (por ejemplo, celda B2)
        imagen = Image(ruta_imagen)
        hoja_nueva.add_image(imagen, 'B2')

# Guardar el libro de Excel con las imágenes
ruta_libro_excel = 'Content\Graficos.xlsx'
libro_excel.save(ruta_libro_excel)


print(f'Imágenes insertadas en "{ruta_libro_excel}" correctamente.')
